In [64]:
import json
import os, sys
from sys import modules
from tqdm.notebook import tqdm
import spacy
from nltk.translate.bleu_score import corpus_bleu
import numpy as np
import random
import importlib

import requests
import urllib

import torch

from typing import Optional, Dict
from dataclasses import dataclass, field
from pydantic import BaseModel
import os
from contextlib import nullcontext
from transformers.hf_argparser import HfArgumentParser
from transformers.models.auto import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM
from fastapi import FastAPI, HTTPException
from uvicorn import run
from sqlite3 import connect, OperationalError


In [2]:
from SpeakQL.Allennlp_models.utils.spider import process_sql, evaluation
from SpeakQL.Allennlp_models.utils.misc_utils import EvaluateSQL, EvaluateSQL_full, \
    Postprocess_rewrite_seq, Postprocess_rewrite_seq_freeze_POS, Postprocess_rewrite_seq_modify_POS

In [28]:
%cd /Users/mac/Desktop/syt/Deep-Learning/Repos/picard

/Users/mac/Desktop/syt/Deep-Learning/Repos/picard


In [33]:
from seq2seq.utils.pipeline import Text2SQLGenerationPipeline, Text2SQLInput, get_schema
from seq2seq.utils.picard_model_wrapper import PicardArguments, PicardLauncher, with_picard
from seq2seq.utils.dataset import DataTrainingArguments
from seq2seq.serve_seq2seq import BackendArguments

In [5]:
nlp = spacy.load('en_core_web_sm')

In [10]:
# del modules['SpeakQL.Allennlp_models.utils.misc_utils']
# del EvaluateSQL, EvaluateSQL_full, Postprocess_rewrite_seq
# from SpeakQL.Allennlp_models.utils.misc_utils import EvaluateSQL, EvaluateSQL_full, Postprocess_rewrite_seq

## Load Picard

In [10]:
%cd /Users/mac/Desktop/syt/Deep-Learning/Repos/picard/seq2seq

/Users/mac/Desktop/syt/Deep-Learning/Repos/picard/seq2seq


In [41]:
base_dir = "/Users/mac/Desktop/syt/Deep-Learning/Repos/picard"

args = {
    "model_path": "tscholak/3vnuv1vf",
    "source_prefix": "",
    "schema_serialization_type": "peteshaw",
    "schema_serialization_randomized": False,
    "schema_serialization_with_db_id": True,
    "schema_serialization_with_db_content": True,
    "normalize_query": True,
    "target_with_db_id": True,
    "db_path": os.path.join(base_dir, "database"),
    "cache_dir": os.path.join(base_dir, "transformers_cache"),
    "num_beams": 4,
    "use_picard": False,
    "launch_picard": True,
    "picard_mode": "parse_with_guards",
    "picard_schedule": "incremental",
    "picard_max_tokens_to_check": 2,
    "device": -1
}

In [35]:
backend_args = BackendArguments()
for k in backend_args.__dict__:
    if k in args:
        backend_args.__setattr__(k, args[k])
backend_args.__dict__

{'model_path': 'tscholak/3vnuv1vf',
 'cache_dir': '/Users/mac/Desktop/syt/Deep-Learning/Repos/picard/transformers_cache',
 'db_path': '/Users/mac/Desktop/syt/Deep-Learning/Repos/picard/database',
 'host': '0.0.0.0',
 'port': 8000,
 'device': -1}

In [36]:
data_training_args = DataTrainingArguments()
for k in data_training_args.__dict__:
    if k in args:
        data_training_args.__setattr__(k, args[k])
data_training_args.__dict__

{'overwrite_cache': False,
 'preprocessing_num_workers': None,
 'max_source_length': 512,
 'max_target_length': 512,
 'val_max_target_length': 512,
 'val_max_time': None,
 'max_train_samples': None,
 'max_val_samples': None,
 'num_beams': 4,
 'num_beam_groups': 1,
 'diversity_penalty': None,
 'ignore_pad_token_for_loss': True,
 'source_prefix': '',
 'schema_serialization_type': 'peteshaw',
 'schema_serialization_randomized': False,
 'schema_serialization_with_db_id': True,
 'schema_serialization_with_db_content': True,
 'normalize_query': True,
 'target_with_db_id': True}

In [42]:
picard_args = PicardArguments()
for k in picard_args.__dict__:
    if k in args:
        picard_args.__setattr__(k, args[k])
picard_args.__dict__

{'use_picard': False,
 'launch_picard': True,
 'picard_host': 'localhost',
 'picard_port': 9090,
 'picard_mode': 'parse_with_guards',
 'picard_schedule': 'incremental',
 'picard_max_tokens_to_check': 2}

In [38]:
config = AutoConfig.from_pretrained(
    backend_args.model_path,
    cache_dir=backend_args.cache_dir,
    max_length=data_training_args.max_target_length,
    num_beams=data_training_args.num_beams,
    num_beam_groups=data_training_args.num_beam_groups,
    diversity_penalty=data_training_args.diversity_penalty,
)

tokenizer = AutoTokenizer.from_pretrained(
    backend_args.model_path,
    cache_dir=backend_args.cache_dir,
    use_fast=True,
)

In [43]:
## For now, not able to set picard = True ("Picard not available") 
if picard_args.use_picard:
    model_cls_wrapper = lambda model_cls: with_picard(
        model_cls=model_cls, picard_args=picard_args, tokenizer=tokenizer
    )
else:
    model_cls_wrapper = lambda model_cls: model_cls

In [44]:
model = model_cls_wrapper(AutoModelForSeq2SeqLM).from_pretrained(
    backend_args.model_path,
    config=config,
    cache_dir=backend_args.cache_dir,
)

In [45]:
pipe = Text2SQLGenerationPipeline(
    model=model,
    tokenizer=tokenizer,
    db_path=backend_args.db_path,
    prefix=data_training_args.source_prefix,
    normalize_query=data_training_args.normalize_query,
    schema_serialization_type=data_training_args.schema_serialization_type,
    schema_serialization_with_db_id=data_training_args.schema_serialization_with_db_id,
    schema_serialization_with_db_content=data_training_args.schema_serialization_with_db_content,
    device=backend_args.device,
)

## Picard predicting

In [50]:
def Picard_predict(question, db_id):
    outputs = pipe(inputs=Text2SQLInput(utterance=question, db_id=db_id))
    output = outputs[0]['generated_text']
    return output

In [51]:
Picard_predict("how many singers do we have?", "concert_singer")

'select count(*) from singer'

In [52]:
Picard_predict("display the employee i D and salary of all employees who report to pye um, first name.", "hr_1")

"select employee_id, salary from employees where manager_id = (select employee_id from employees where first_name = 'pye' )"

In [53]:
Picard_predict('find the weight of the youngest dog.', 'pets_1')

'select weight from pets where pettype = "dog" order by age asc limit 1'

## Predict & Evaluate SQL given rewriter output / original / ASR cands

In [54]:
EvaluateSQL(pred_str='SELECT * FROM singer WHERE name = "Joe Sharp"',
            gold_str='SELECT * FROM singer WHERE name = "DEF"',
            db='concert_singer'), \
EvaluateSQL(pred_str='SELECT country FROM singer WHERE name = "ABC"',
            gold_str='SELECT country FROM singer WHERE name = "DEF" ORDER BY age LIMIT 1',
            db='concert_singer')

((1, 1.0, 0), (0, 0.5, 1))

### Basic compares

In [55]:
# test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter+phonemes.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'

In [56]:
# # human test 
# test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
# orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

In [57]:
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

(547, 3075, 1034)

In [58]:
# Just using the 1st ASR candidate, no correction 

ref_list = []
hyp_list = []

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]

    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _db_id = o['db_id']
    
    _pred_sql = Picard_predict(c['question'], _db_id)
    
    _gold_sql = c['query']
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
    
    c['pred_sql'] = _pred_sql
    c['score'] = _score
    c['exact'] = _exact
    c['exec'] = _exec

    _question_toks = [_t.lower() for _t in c['question_toks']]
    _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]
    ref_list.append([_gold_question_toks])
    hyp_list.append(_question_toks)


  0%|          | 0/547 [00:00<?, ?it/s]

select weight from pets where pettype = "dog" order by age asc limit 1
SELECT weight FROM pets ORDER BY pet_age LIMIT 1
pets_1
process_sql.get_sql() failed
select count(distinct t2.petid) from has_pet as t1 join pets as t2 on t1.petid = t2.petid join student as t3 on t1.stuid = t3.stuid where t3.h > 20
SELECT count(*) FROM student AS T1 JOIN has_pet AS T2 ON T1.stuid  =  T2.stuid WHERE T1.age  >  20
pets_1
process_sql.get_sql() failed
select t1.fname from student as t1 join has_pet as t2 on t1.stuid = t2.stuid join pets as t3 on t2.petid = t3.petid where t3.pettype = "Captain Doug" intersect select t1.fname from student as t1 join has_pet as t2 on t1.stuid = t2.stuid join pets as t3 on t2.petid = t3.petid where t3.pettype = "Select t3.pettype from pets as t3 on t2.petid = t3.petid where t3.pettype = "Select t3.pettype from pets as t4 on t2.petid = t4.petid where t3.pettype = "Select t3.pettype from pets as t4 on t2.petid = t4.petid where t3.pettype = "Select t
SELECT T1.Fname FROM stud

select t2.paragraph_text, t2.paragraph_id, t3.document_name from documents as t1 join paragraphs as t2 on t1.document_id = t2.document_id join paragraphs as t3 on t2.paragraph_id = t3.paragraph_id where t1.document_name = "Welcome to NY"
SELECT T1.paragraph_id ,   T1.paragraph_text FROM Paragraphs AS T1 JOIN Documents AS T2 ON T1.document_id  =  T2.document_id WHERE T2.Document_Name  =  'Welcome to NY'
cre_Doc_Template_Mgt
process_sql.get_sql() failed
select avg(t1.odds) from documents as t1 join paragraphs as t2 on t1.document_id = t2.document_id where t2.paragraph_text = "Brazil" intersect select avg(t1.odds) from documents as t1 join paragraphs as t2 on t1.document_id = t2.document_id where t2.paragraph_text = "Ireland"
SELECT document_id FROM Paragraphs WHERE paragraph_text  =  'Brazil' INTERSECT SELECT document_id FROM Paragraphs WHERE paragraph_text  =  'Ireland'
cre_Doc_Template_Mgt
process_sql.get_sql() failed
select agent from teacher
SELECT Age ,  Hometown FROM teacher
course

select t1.area_code from area_code_state as t1 join votes as t2 on t1.area_code = t2.area_code group by t1.area_code order by count(*) desc limit 1
SELECT T1.area_code FROM area_code_state AS T1 JOIN votes AS T2 ON T1.state  =  T2.state GROUP BY T1.area_code ORDER BY count(*) DESC LIMIT 1
voter_1
process_sql.get_sql() failed
select t1.area_code from area_code_state as t1 join votes as t2 on t1.area_code = t2.area_code join contestants as t3 on t2.contestant_number = t3.contestant_number where t3.contestant_name = 'Tabatha' intersect select t1.area_code from area_code_state as t1 join votes as t2 on t1.area_code = t2.area_code join contestants as t3 on t2.contestant_number = t3.contestant_number where t3.contestant_name = 'Kelly Kloss'
SELECT T3.area_code FROM contestants AS T1 JOIN votes AS T2 ON T1.contestant_number  =  T2.contestant_number JOIN area_code_state AS T3 ON T2.state  =  T3.state WHERE T1.contestant_name  =  'Tabatha Gehling' INTERSECT SELECT T3.area_code FROM contestants 

select name from country where countrycode not in (select countrycode from countrylanguage where language = 'English') and governmentform != 'Republic'
SELECT Code FROM country WHERE GovernmentForm != "Republic" EXCEPT SELECT CountryCode FROM countrylanguage WHERE LANGUAGE  =  "English"
world_1
process_sql.get_sql() failed
select t1.name from city as t1 join countrylanguage as t2 on t1.countrycode = t2.countrycode where t2.continent = 'Europe' and t2.isofficial != 'English'
SELECT DISTINCT T2.Name FROM country AS T1 JOIN city AS T2 ON T2.CountryCode  =  T1.Code WHERE T1.Continent  =  'Europe' AND T1.Name NOT IN (SELECT T3.Name FROM country AS T3 JOIN countrylanguage AS T4 ON T3.Code  =  T4.CountryCode WHERE T4.IsOfficial  =  'T' AND T4.Language  =  'English')
world_1
process_sql.get_sql() failed
select distinct t1.name from city as t1 join countrylanguage as t2 on t1.countrycode = t2.countrycode where t2.continent = "Asia" and t2.language = "Chinese"
SELECT DISTINCT T3.Name FROM countr

In [ ]:
c['question'], _db_id

In [59]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
_avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
_avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

# ## Std-dev (1st cand only)
# _std_1st = np.std([d[0]['score'] for d in test_dataset])

## BLEU 
_bleu = corpus_bleu(list_of_references=ref_list,
                    hypotheses=hyp_list)

# print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg = {:.4f}'.format(_avg_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))
print(f'BLEU = {_bleu:.4f}')


avg = 0.6884
avg_exact = 0.5539
avg_exec = 0.5686
BLEU = 0.8010


In [ ]:
# EvaluateSQL_full(plist=[d[0]['pred_sql'] for d in test_dataset],
#                  glist=[d[0]['query'] for d in test_dataset],
#                  db_id_list=[d[0]['db_id'] for d in test_dataset],
#                  etype='all')

In [60]:
# Dump the predictions 
output_dir = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs'
# Orig
picard_out_dir = os.path.join(output_dir, 'picard-test-save')
os.makedirs(picard_out_dir, exist_ok=True)
test_output_path = os.path.join(picard_out_dir, 'First-cands.json')

# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/Assembly/First-cands.json'
# ASR
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/First-cands.json'
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/Assembly/First-cands.json'
# Mixed
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/First-cands.json'
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/Assembly/First-cands.json'

with open(test_output_path, 'w') as f:
    json.dump(test_dataset, f, indent=4)

In [61]:
# Using original text (no ASR)

with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

(547, 3075, 1034)

In [62]:
# Using original text (no ASR)

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
        
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _db_id = o['db_id']
    
    _pred_sql = Picard_predict(c['gold_question'], _db_id)
    
    _gold_sql = c['query']
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
    
    c['pred_sql'] = _pred_sql
    c['score'] = _score
    c['exact'] = _exact
    c['exec'] = _exec


  0%|          | 0/547 [00:00<?, ?it/s]

select weight from pets where pettype = "dog" order by age asc limit 1
SELECT weight FROM pets ORDER BY pet_age LIMIT 1
pets_1
process_sql.get_sql() failed
select weight from pets order by age asc limit 1
SELECT weight FROM pets ORDER BY pet_age LIMIT 1
pets_1
process_sql.get_sql() failed
select major, age from student where stuid not in (select stuid from has_pet where pettype = "cat")
SELECT major ,  age FROM student WHERE stuid NOT IN (SELECT T1.stuid FROM student AS T1 JOIN has_pet AS T2 ON T1.stuid  =  T2.stuid JOIN pets AS T3 ON T3.petid  =  T2.petid WHERE T3.pettype  =  'cat')
pets_1
process_sql.get_sql() failed
select stuid from student except select stuid from has_pet where pettype = "cat"
SELECT stuid FROM student EXCEPT SELECT T1.stuid FROM student AS T1 JOIN has_pet AS T2 ON T1.stuid  =  T2.stuid JOIN pets AS T3 ON T3.petid  =  T2.petid WHERE T3.pettype  =  'cat'
pets_1
process_sql.get_sql() failed
select stuid from student except select stuid from has_pet where pettype = "

select t1.first_name, t1.country_code from players as t1 join matches as t2 on t1.player_id = t2.winner_id where t2.tournament_name = "WTA Championships" intersect select t1.first_name, t1.country_code from players as t1 join matches as t2 on t1.player_id = t2.winner_id where t2.tournament_name = "Australian Open"
SELECT T1.country_code ,  T1.first_name FROM players AS T1 JOIN matches AS T2 ON T1.player_id  =  T2.winner_id WHERE T2.tourney_name  =  'WTA Championships' INTERSECT SELECT T1.country_code ,  T1.first_name FROM players AS T1 JOIN matches AS T2 ON T1.player_id  =  T2.winner_id WHERE T2.tourney_name  =  'Australian Open'
wta_1
process_sql.get_sql() failed
select t1.winner_name, t1.winner_rank_points from winners as t1 join rankings as t2 on t1.winner_id = t2.player_id group by t2.player_id order by count(*) desc limit 1
SELECT winner_name ,  winner_rank_points FROM matches GROUP BY winner_name ORDER BY count(*) DESC LIMIT 1
wta_1
process_sql.get_sql() failed
select t1.winner_n

select count(distinct t1.continent) from country as t1 join countrylanguage as t2 on t1.countrycode = t2.countrycode where t2.language = "Chinese"
SELECT COUNT( DISTINCT Continent) FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code  =  T2.CountryCode WHERE T2.Language  =  "Chinese"
world_1
process_sql.get_sql() failed
select distinct t1.region from country as t1 join countrylanguage as t2 on t1.countrycode = t2.countrycode where t2.language = "English" or t2.language = "Dutch"
SELECT DISTINCT T1.Region FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code  =  T2.CountryCode WHERE T2.Language  =  "English" OR T2.Language  =  "Dutch"
world_1
process_sql.get_sql() failed
select distinct t1.region from country as t1 join countrylanguage as t2 on t1.countrycode = t2.countrycode where t2.language = "Dutch" or t2.language = "English"
SELECT DISTINCT T1.Region FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code  =  T2.CountryCode WHERE T2.Language  =  "English" OR T2.Language  = 

In [63]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
_avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
_avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

# ## Std-dev (1st cand only)
# _std_1st = np.std([d[0]['score'] for d in test_dataset])

# print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg = {:.4f}'.format(_avg_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))


avg = 0.7874
avg_exact = 0.6929
avg_exec = 0.7148


In [ ]:
EvaluateSQL_full(plist=[d[0]['pred_sql'] for d in test_dataset],
                 glist=[d[0]['query'] for d in test_dataset],
                 db_id_list=[d[0]['db_id'] for d in test_dataset],
                 etype='all')

In [ ]:
random_subset = random.sample(test_dataset, k=100)

EvaluateSQL_full(plist=[d[0]['pred_sql'] for d in random_subset],
                 glist=[d[0]['query'] for d in random_subset],
                 db_id_list=[d[0]['db_id'] for d in random_subset],
                 etype='all')

## Old

### Variations

#### End2end

In [205]:
VERSION = '4.1.0.1_laststep'

end2end_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
# test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_reranker.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'


In [206]:
with open(end2end_pred_path, 'r') as f:
    end2end_preds = [json.loads(l) for l in f]
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)
    
len(end2end_preds), len(orig_dev_dataset)

(3075, 1034)

In [207]:
# Quick evaluation: only using the 1st ASR candidate

_seen_ids = set()
_first_cand_preds = []

for i, p in tqdm(enumerate(end2end_preds), total=len(end2end_preds)):

    # p = rewriter_ILM_preds[pred_idx]
    # _o_idx = c['original_id']
    # o = orig_dev_dataset[_o_idx]
    # assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
    # assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _o_idx = p['original_id']
    if _o_idx in _seen_ids:
        continue
    else:
        _seen_ids.add(_o_idx)
        
    o = orig_dev_dataset[_o_idx]
    
    # Debug 
    # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

    _db_id = o['db_id']
    _pred_sql = p['pred_sql']
    _gold_sql = p['gold_sql']
    assert _gold_sql == o['query'], (_gold_sql, o['query'])
    
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
    
    # Save prediction results 
    p['score'] = _score
    p['exact'] = _exact
    p['exec'] = _exec
    
    _first_cand_preds.append(p)
    

SELECT Max(performance.Official_ratings_(millions)), Min(performance.Share) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed
SELECT Max(performance.Official_ratings_(millions)), Min(performance.Share) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed



In [208]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([p['score'] for p in _first_cand_preds]) / len(_first_cand_preds)
_avg_exact_1st = sum([p['exact'] for p in _first_cand_preds]) / len(_first_cand_preds)
_avg_exec_1st = sum([p['exec'] for p in _first_cand_preds]) / len(_first_cand_preds)

## Std-dev (1st cand only)
# _std_1st = np.std([c['score'] for d in test_dataset for c in d if c['is_reranker_selection']])

print('avg = {:.4f}'.format(_avg_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))


avg = 0.7660
avg_exact = 0.4973
avg_exec = 0.3620


#### Reranker

In [91]:
VERSION = '1.14.1.2'
HUMAN_TEST = False

if not HUMAN_TEST:
    reranker_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_reranker.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
else:
    reranker_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_reranker.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'


In [ ]:
## Aggreg with rewriter cands 

RERANKER_VERSION = '1.10.0.2'
REWRITER_VERSION = '2.6.0.2t-2.6.0.2i'
HUMAN_TEST = True

if not HUMAN_TEST:
    reranker_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/aggreg/output-{RERANKER_VERSION}-with-{REWRITER_VERSION}.json'
    test_dataset_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/aggreg_extra_cands/test_reranker_with_{REWRITER_VERSION}.json'
    orig_dev_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
else:
    reranker_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/aggreg/output-humantest-yshao-{RERANKER_VERSION}-with-{REWRITER_VERSION}.json'
    test_dataset_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/aggreg_extra_cands/human_test_yshao_reranker_with_{REWRITER_VERSION}.json'
    orig_dev_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'
    

In [92]:
with open(reranker_pred_path, 'r') as f:
    reranker_preds = [json.loads(l) for l in f.readlines()]
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(reranker_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

(3075, 547, 3075, 1034)

In [93]:
reranker_preds[0].keys(), test_dataset[0][0].keys()

(dict_keys(['score_preds', 'question', 'original_id']),
 dict_keys(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql', 'span_ranges', 'original_id', 'ratsql_pred_sql', 'gold_question', 'gold_question_toks', 'ratsql_pred_exact', 'ratsql_pred_score', 'question_toks_edit_distance']))

In [94]:
pred_idx = 0

ref_list = []
hyp_list = []

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
    
    # Find the cand with highest score 
    d_preds = []
    for _ in d:
        p = reranker_preds[pred_idx]
        d_preds.append(p)
        pred_idx += 1
    
    _c_idx = np.argmax([p['score_preds'] for p in d_preds])
    
    c = d[_c_idx]
    p = d_preds[_c_idx]
    
    for _c in d:
        _c['is_reranker_selection'] = False
    c['is_reranker_selection'] = True
    
    # Use the selected cand to proceed 
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _db_id = o['db_id']

    _question = c['question']

    if c['ratsql_pred_sql'] is not None:
        # If SQL is already predicted, no need to predict again 
        _pred_sql = c['ratsql_pred_sql']
    else:
        # If SQL not yet predicted, do it 
        _pred_sql = Question(_question, _db_id)[0]['inferred_code']

    _gold_sql = c['query']
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

#     # Save the taggerILM raw outputs, for later aggregation 
#     c['pred_tags'] = p['rewriter_tags']
#     c['pred_ILM'] = p['rewrite_seq_prediction']
#     c['pred_ILM_cands'] = p['rewrite_seq_prediction_cands']
    
    # Save prediction results 
    # c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
    c['pred_sql'] = p['pred_sql'] = _pred_sql
    # p['gold_sql'] = _gold_sql
    c['score'] = p['score'] = _score
    c['exact'] = p['exact'] = _exact
    c['exec'] = p['exec'] = _exec

    _question_toks = [_t.lower() for _t in c['question_toks']]
    _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

    ref_list.append([_gold_question_toks])
    hyp_list.append(_question_toks)


In [95]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([c['score'] for d in test_dataset for c in d if c['is_reranker_selection']]) / len(test_dataset)
_avg_exact_1st = sum([c['exact'] for d in test_dataset for c in d if c['is_reranker_selection']]) / len(test_dataset)
_avg_exec_1st = sum([c['exec'] for d in test_dataset for c in d if c['is_reranker_selection']]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([c['score'] for d in test_dataset for c in d if c['is_reranker_selection']])

## BLEU 
_bleu = corpus_bleu(list_of_references=ref_list,
                    hypotheses=hyp_list)

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))
print(f'BLEU = {_bleu:.4f}')



avg = 0.7599 (std = 0.2977)
avg_exact = 0.5027
avg_exec = 0.3620
BLEU = 0.7853


In [ ]:
EvaluateSQL_full(plist=[c['pred_sql'] for d in test_dataset for c in d if c['is_reranker_selection']],
                 glist=[d[0]['query'] for d in test_dataset],
                 db_id_list=[d[0]['db_id'] for d in test_dataset])

In [96]:
# Non ASR
if not HUMAN_TEST:
    test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
else:
    test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}.json'
    
# ASR
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/{VERSION}.json'

with open(test_output_path, 'w') as f:
    json.dump(test_dataset, f, indent=4)

#### Tagger-ILM

In [214]:
## Tagger accuracy 
VERSION_LIST = [f'2.16.0.{v}t' for v in range(5)]

for VERSION in VERSION_LIST:
    predict_fname = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{VERSION}.json'

    with open(predict_fname, 'r') as f:
        predicts = [json.loads(l) for l in f]
    len(predicts)

    correct_tags = 0
    total_tags = 0

    non_O_matches = 0
    non_O_preds = 0
    non_O_golds = 0

    for p in predicts:
        _pred_tags = p['tags_prediction']
        _gold_tags = [t for t in p['gold_tags'] if t != 'O']
        assert len(_pred_tags) == len(_gold_tags)

        _cor = sum([_p == _g for _p, _g in zip(_pred_tags, _gold_tags)])
        _total = len(_pred_tags)
        assert -1e-8 < _cor / _total - p['tags_accuracy'] < 1e-8, f"{_cor / _total}, {p['tags_accuracy']}"

        for _p, _g in zip(_pred_tags, _gold_tags):
            if _p != 'O-KEEP':
                non_O_preds += 1
            if _g != 'O-KEEP':
                non_O_golds += 1
            if _p == _g and _p != 'O-KEEP':
                non_O_matches += 1

        correct_tags += _cor
        total_tags += _total

    tagger_prec = non_O_matches / non_O_preds
    tagger_recall = non_O_matches / non_O_golds
    tagger_F1 = 2 * tagger_prec * tagger_recall / (tagger_prec + tagger_recall)

    print(f'VERSION: {VERSION}')
    print(f'Overall tagger accuracy = {correct_tags}/{total_tags} = {correct_tags / total_tags:.4f}')
    print(f'Tagger P = {tagger_prec:.4f}, R = {tagger_recall:.4f}, F1 = {tagger_F1:.4f}')
    print()
    

VERSION: 2.16.0.0t
Overall tagger accuracy = 38814/41776 = 0.9291
Tagger P = 0.7764, R = 0.6817, F1 = 0.7260

VERSION: 2.16.0.1t
Overall tagger accuracy = 38782/41776 = 0.9283
Tagger P = 0.7529, R = 0.6967, F1 = 0.7237

VERSION: 2.16.0.2t
Overall tagger accuracy = 38742/41776 = 0.9274
Tagger P = 0.7446, R = 0.6858, F1 = 0.7140

VERSION: 2.16.0.3t
Overall tagger accuracy = 38893/41776 = 0.9310
Tagger P = 0.7663, R = 0.6974, F1 = 0.7303

VERSION: 2.16.0.4t
Overall tagger accuracy = 38932/41776 = 0.9319
Tagger P = 0.7640, R = 0.7072, F1 = 0.7345



In [211]:
## Full tagger-ILM

HUMAN_TEST = False
ASR = 'Amazon'

In [251]:
VERSION_LIST = [f'2.12.1.{v}t-2.21.0.{v}i' for v in [3,4]]
VERSION_LIST += [f'2.12.1.{v}t-2.22.1.{v}i' for v in [3,4]]
# VERSION_LIST = [f'2.12.1.{v}t-2.16.{i}.{v}i' for i in [3,4] for v in range(5)]

for VERSION in VERSION_LIST:
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/output-{}.json'.format(VERSION)
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
        else:
            rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

    with open(rewriter_ILM_pred_path, 'r') as f:
        rewriter_ILM_preds = [json.loads(l) for l in f.readlines()]
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
    with open(orig_dev_path, 'r') as f:
        orig_dev_dataset = json.load(f)

    # len(rewriter_ILM_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

    ## Quick evaluation: only using the 1st ASR candidate

    pred_idx = 0

    ref_list = []
    hyp_list = []

    for d in tqdm(test_dataset, desc=f'VERSION {VERSION}'):
        if len(d) == 0:
            continue

        c = d[0]

        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

        # Debug 
        # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

        _db_id = o['db_id']

        # _tags = p['tags_prediction']  # For previous taggerILM joint model 
        # _tags = p['tags']  # Before adding align_tags (when 'tags' refers to 'rewriter_tags')

        _tags = p['rewriter_tags']
        _rewrite_seq = p['rewrite_seq_prediction']
        _question_toks = c['question_toks']

        _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        _rewritten_question = ' '.join(_rewritten_question_toks)

        _pred_sql = Question(_rewritten_question, _db_id, model_dict=model_dicts['orig'])[0]['inferred_code']

        _gold_sql = c['query']
        _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

        # Save the taggerILM raw outputs, for later aggregation 
        c['pred_tags'] = p['rewriter_tags']
        c['pred_ILM'] = p['rewrite_seq_prediction']
        c['pred_ILM_cands'] = p['rewrite_seq_prediction_cands']

        # Save prediction results 
        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score
        c['exact'] = p['exact'] = _exact
        c['exec'] = p['exec'] = _exec

        _rewritten_question_toks = [_t.lower() for _t in _rewritten_question_toks]
        _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

        ref_list.append([_gold_question_toks])
        hyp_list.append(_rewritten_question_toks)

        pred_idx += len(d)

    # Only using the 1st candidate to rewrite 
    _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
    _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
    _avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

    ## Std-dev (1st cand only)
    _std_1st = np.std([d[0]['score'] for d in test_dataset])

    ## BLEU 
    _bleu = corpus_bleu(list_of_references=ref_list,
                        hypotheses=hyp_list)

    print('='*20, f'VERSION: {VERSION}', '='*20)
    print('avg_exact = {:.4f}'.format(_avg_exact_1st))
    # print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
    print('avg = {:.4f}'.format(_avg_1st))
    print('avg_exec = {:.4f}'.format(_avg_exec_1st))
    print(f'BLEU = {_bleu:.4f}')
    print('='*55)
    
    # Non ASR
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/ratsql-test-save/{VERSION}.json'
        else:
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
    else:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}.json'
    # ASR
    # test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/{VERSION}.json'
    # Mixed
    # test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/{VERSION}.json'

    with open(test_output_path, 'w') as f:
        json.dump(test_dataset, f, indent=4)


--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'U-EDIT', 'U-DEL', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['ids'], ["'GV", "'"], ["'SF", "'"], ['?']]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'U-EDIT', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

--- Not enough edits ---
Tags: ['O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: []
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'U-EDIT', 'U-DEL', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['and'], ["'chervil", "'"]]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['code'], ['``', 'CWS', "''"]]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'U-DEL', 'U-EDIT', 'O-KEEP', 'O-KEEP', 

--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'U-DEL', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['at']]
--- Not enough edits ---
Tags: ['U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['Retrieve', 'all']]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'U-EDIT', 'U-DEL', 'U-EDIT', 'O', 'O', 'O', 'O', '

In [ ]:
# # Non ASR
# if not HUMAN_TEST:
#     if ASR == 'AssemblyAI':
#         test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/ratsql-test-save/{VERSION}.json'
#     else:
#         test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
# else:
#     test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}.json'
# # ASR
# # test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/{VERSION}.json'
# # Mixed
# # test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/{VERSION}.json'

# with open(test_output_path, 'w') as f:
#     json.dump(test_dataset, f, indent=4)

##### Freeze/Modify POS

In [246]:
VERSION_LIST = [f'2.12.1.{v}t-2.12.3.{v}i' for v in [3]]
# VERSION_LIST += [f'2.12.1.{v}t-2.16.5.{v}i' for v in [3,4]]
# VERSION_LIST = [f'2.12.1.{v}t-2.16.{i}.{v}i' for i in [3,4] for v in range(5)]

# POS_LIST = ["PUNCT", "NUM", "CCONJ", "PRON", "AUX", "VERB", "NOUN", "DET", "ADP", "ADJ", "PROPN"]
POS_LIST = ["SCONJ", "PART", "ADV"]

GROUP_LIST = [(_v, _p) for _v in VERSION_LIST for _p in POS_LIST]

for VERSION, POS in GROUP_LIST:
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/output-{}.json'.format(VERSION)
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
        else:
            rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

    with open(rewriter_ILM_pred_path, 'r') as f:
        rewriter_ILM_preds = [json.loads(l) for l in f.readlines()]
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
    with open(orig_dev_path, 'r') as f:
        orig_dev_dataset = json.load(f)

    # len(rewriter_ILM_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

    ## Quick evaluation: only using the 1st ASR candidate

    pred_idx = 0

    ref_list = []
    hyp_list = []

    for d in tqdm(test_dataset, desc=f'VERSION {VERSION}, POS {POS}'):
        if len(d) == 0:
            continue

        c = d[0]

        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

        # Debug 
        # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

        _db_id = o['db_id']

        # _tags = p['tags_prediction']  # For previous taggerILM joint model 
        # _tags = p['tags']  # Before adding align_tags (when 'tags' refers to 'rewriter_tags')

        _tags = p['rewriter_tags']
        _rewrite_seq = p['rewrite_seq_prediction']
        _question_toks = c['question_toks']

        _rewritten_question_toks = Postprocess_rewrite_seq_freeze_POS(
            _tags,
            _rewrite_seq,
            _question_toks,
            freeze_POS=POS,
            nlp=nlp)
        _rewritten_question = ' '.join(_rewritten_question_toks)

        _pred_sql = Question(_rewritten_question, _db_id, model_dict=model_dicts['orig'])[0]['inferred_code']

        _gold_sql = c['query']
        _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

        # Save the taggerILM raw outputs, for later aggregation 
        c['pred_tags'] = p['rewriter_tags']
        c['pred_ILM'] = p['rewrite_seq_prediction']
        c['pred_ILM_cands'] = p['rewrite_seq_prediction_cands']

        # Save prediction results 
        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score
        c['exact'] = p['exact'] = _exact
        c['exec'] = p['exec'] = _exec

        _rewritten_question_toks = [_t.lower() for _t in _rewritten_question_toks]
        _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

        ref_list.append([_gold_question_toks])
        hyp_list.append(_rewritten_question_toks)

        pred_idx += len(d)

    # Only using the 1st candidate to rewrite 
    _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
    _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
    _avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

    ## Std-dev (1st cand only)
    _std_1st = np.std([d[0]['score'] for d in test_dataset])

    ## BLEU 
    _bleu = corpus_bleu(list_of_references=ref_list,
                        hypotheses=hyp_list)

    print('='*20, f'VERSION: {VERSION}', '='*20)
    print('avg_exact = {:.4f}'.format(_avg_exact_1st))
    # print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
    print('avg = {:.4f}'.format(_avg_1st))
    print('avg_exec = {:.4f}'.format(_avg_exec_1st))
    print(f'BLEU = {_bleu:.4f}')
    print('='*55)
    
    # Non ASR
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/ratsql-test-save/{VERSION}-freeze={POS}.json'
        else:
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}-freeze={POS}.json'
    else:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}-freeze={POS}.json'

    with open(test_output_path, 'w') as f:
        json.dump(test_dataset, f, indent=4)



SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: 2.12.1.3t-2.12.3.3i ====================
avg_exact = 0.5356
avg = 0.7728
avg_exec = 0.3784
BLEU = 0.8746


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: 2.12.1.3t-2.12.3.3i ====================
avg_exact = 0.5356
avg = 0.7728
avg_exec = 0.3784
BLEU = 0.8746



==================== VERSION: 2.12.1.3t-2.12.3.3i ====================
avg_exact = 0.5356
avg = 0.7736
avg_exec = 0.3803
BLEU = 0.8742


In [247]:
VERSION_LIST = [f'2.12.1.{v}t-2.12.3.{v}i' for v in [3]]
# VERSION_LIST += [f'2.12.1.{v}t-2.16.5.{v}i' for v in [3,4]]
# VERSION_LIST = [f'2.12.1.{v}t-2.16.{i}.{v}i' for i in [3,4] for v in range(5)]

# POS_LIST = ["PUNCT", "NUM", "CCONJ", "PRON", "AUX", "VERB", "NOUN", "DET", "ADP", "ADJ", "PROPN"]
POS_LIST = ["SCONJ", "PART", "ADV"]

GROUP_LIST = [(_v, _p) for _v in VERSION_LIST for _p in POS_LIST]

for VERSION, POS in GROUP_LIST:
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/output-{}.json'.format(VERSION)
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
        else:
            rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

    with open(rewriter_ILM_pred_path, 'r') as f:
        rewriter_ILM_preds = [json.loads(l) for l in f.readlines()]
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
    with open(orig_dev_path, 'r') as f:
        orig_dev_dataset = json.load(f)

    # len(rewriter_ILM_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

    ## Quick evaluation: only using the 1st ASR candidate

    pred_idx = 0

    ref_list = []
    hyp_list = []

    for d in tqdm(test_dataset, desc=f'VERSION {VERSION}, POS {POS}'):
        if len(d) == 0:
            continue

        c = d[0]

        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

        # Debug 
        # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

        _db_id = o['db_id']

        # _tags = p['tags_prediction']  # For previous taggerILM joint model 
        # _tags = p['tags']  # Before adding align_tags (when 'tags' refers to 'rewriter_tags')

        _tags = p['rewriter_tags']
        _rewrite_seq = p['rewrite_seq_prediction']
        _question_toks = c['question_toks']

        _rewritten_question_toks = Postprocess_rewrite_seq_modify_POS(
            _tags,
            _rewrite_seq,
            _question_toks,
            modify_POS=POS,
            nlp=nlp)
        _rewritten_question = ' '.join(_rewritten_question_toks)

        _pred_sql = Question(_rewritten_question, _db_id, model_dict=model_dicts['orig'])[0]['inferred_code']

        _gold_sql = c['query']
        _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

        # Save the taggerILM raw outputs, for later aggregation 
        c['pred_tags'] = p['rewriter_tags']
        c['pred_ILM'] = p['rewrite_seq_prediction']
        c['pred_ILM_cands'] = p['rewrite_seq_prediction_cands']

        # Save prediction results 
        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score
        c['exact'] = p['exact'] = _exact
        c['exec'] = p['exec'] = _exec

        _rewritten_question_toks = [_t.lower() for _t in _rewritten_question_toks]
        _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

        ref_list.append([_gold_question_toks])
        hyp_list.append(_rewritten_question_toks)

        pred_idx += len(d)

    # Only using the 1st candidate to rewrite 
    _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
    _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
    _avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

    ## Std-dev (1st cand only)
    _std_1st = np.std([d[0]['score'] for d in test_dataset])

    ## BLEU 
    _bleu = corpus_bleu(list_of_references=ref_list,
                        hypotheses=hyp_list)

    print('='*20, f'VERSION: {VERSION}', '='*20)
    print('avg_exact = {:.4f}'.format(_avg_exact_1st))
    # print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
    print('avg = {:.4f}'.format(_avg_1st))
    print('avg_exec = {:.4f}'.format(_avg_exec_1st))
    print(f'BLEU = {_bleu:.4f}')
    print('='*55)
    
    # Non ASR
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/ratsql-test-save/{VERSION}-modify={POS}.json'
        else:
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}-modify={POS}.json'
    else:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}-modify={POS}.json'

    with open(test_output_path, 'w') as f:
        json.dump(test_dataset, f, indent=4)





==================== VERSION: 2.12.1.3t-2.12.3.3i ====================
avg_exact = 0.4570
avg = 0.7273
avg_exec = 0.3400
BLEU = 0.8010



==================== VERSION: 2.12.1.3t-2.12.3.3i ====================
avg_exact = 0.4570
avg = 0.7273
avg_exec = 0.3400
BLEU = 0.8010


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: 2.12.1.3t-2.12.3.3i ====================
avg_exact = 0.4552
avg = 0.7258
avg_exec = 0.3382
BLEU = 0.8014


#### Loading from predicted file (only 1st cand is predicted!)

In [ ]:
VERSION = '3.6.0.2'

test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

with open(test_output_path, 'r') as f:
    test_dataset = json.load(f)

len(test_dataset), test_dataset[0][0].keys()

In [ ]:
# Using EvaluateSQL_full 

# tables_json = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/tables.json'
# kmaps = evaluation.build_foreign_key_map_from_json(tables_json)

plist = [d[0]['pred_sql'] for d in test_dataset]
glist = [d[0]['query'] for d in test_dataset]
db_id_list = [d[0]['db_id'] for d in test_dataset]

EvaluateSQL_full(glist=glist,
                 plist=plist,
                 db_id_list=db_id_list,
                 kmaps=kmaps)


In [ ]:
# For reranker output file 
VERSION = '1.10.0.2'

test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

with open(test_output_path, 'r') as f:
    test_dataset = json.load(f)

len(test_dataset), test_dataset[0][0].keys()

In [ ]:
plist = [c['ratsql_pred_sql'] for d in test_dataset for c in d if c['is_reranker_selection'] == 1]
glist = [d[0]['query'] for d in test_dataset]
db_id_list = [d[0]['db_id'] for d in test_dataset]
assert len(plist) == len(glist) == len(db_id_list)

EvaluateSQL_full(glist=glist,
                 plist=plist,
                 db_id_list=db_id_list,
                 kmaps=kmaps)

In [ ]:
plist = [d[0]['ratsql_pred_sql'] for d in test_dataset]
glist = [d[0]['query'] for d in test_dataset]
db_id_list = [d[0]['db_id'] for d in test_dataset]
assert len(plist) == len(glist) == len(db_id_list)

EvaluateSQL_full(glist=glist,
                 plist=plist,
                 db_id_list=db_id_list,
                 kmaps=kmaps)

In [ ]:
# Using all ASR candidates (no longer in use)

pred_idx = 0

for d in tqdm(test_dataset):
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _db_id = o['db_id']

        # _tags = p['tags_prediction']
        _tags = p['tags']
        _rewrite_seq = p['rewrite_seq_prediction']
        _question_toks = c['question_toks']
        
        _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        _rewritten_question = ' '.join(_rewritten_question_toks)
        
        _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
        
        _gold_sql = c['query']
        _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
        
        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score

        pred_idx += 1



In [ ]:
# Using all the candidates to rewrite 
print(sum([p['score'] for p in rewriter_ILM_preds]) / len(rewriter_ILM_preds))
print(sum([c['score'] for d in test_dataset for c in d]) / sum([len(d) for d in test_dataset]))

# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))

In [ ]:
# Evaluation process with oracle tags (no longer in use for version>=2.3.0)

pred_idx = 0

for d in tqdm(test_dataset):
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _db_id = o['db_id']

        _tags = p['gold_tags']
        _rewrite_seq = p['oracle_tags_rewrite_seq_prediction']
        _question_toks = c['question_toks']
        
        _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        _rewritten_question = ' '.join(_rewritten_question_toks)
        
        _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
        
        _gold_sql = c['query']
        _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
        
        c['oracle_tags_rewritten_question'] = p['oracle_tags_rewritten_question'] = _rewritten_question
        c['oracle_tags_pred_sql'] = p['oracle_tags_pred_sql'] = _pred_sql
        c['oracle_tags_score'] = p['oracle_tags_score'] = _score

        pred_idx += 1



In [ ]:
# Using all the candidates to rewrite 
print(sum([p['oracle_tags_score'] for p in rewriter_ILM_preds]) / len(rewriter_ILM_preds))
print(sum([c['oracle_tags_score'] for d in test_dataset for c in d]) / sum([len(d) for d in test_dataset]))

# Only using the 1st candidate to rewrite 
_oracle_avg_1st = sum([d[0]['oracle_tags_score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_oracle_std_1st = np.std([d[0]['oracle_tags_score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_oracle_avg_1st, _oracle_std_1st))

In [ ]:
# Merge results in a single dataset obj 

test_pred_dataset = []

pred_idx = 0

for d in tqdm(test_dataset):
    _pred_d = []
    
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _pred_c = dict()
        
        _pred_c['ASR_question'] = p['question']
        _pred_c['ASR_question_pred_sql'] = c['ratsql_pred_sql']
        
        _pred_c['gold_question'] = c['gold_question']
        # _pred_c['gold_question_pred_sql'] = orig_dev_preds[c['original_id']]
        
        _pred_c['tag_prediction'] = list(zip(p['question'].split(' '), p['tags_prediction']))
        _pred_c['rewrite_seq'] = []
        for t in p['rewrite_seq_prediction']:
            _pred_c['rewrite_seq'].append(t)
            if t == '@end@': break
        _pred_c['rewritten_question'] = p['rewritten_question']
        _pred_c['pred_sql'] = p['pred_sql']
        _pred_c['score'] = p['score']
        
        _pred_c['gold_tags'] = list(zip(p['question'].split(' '), p['gold_tags']))
        _pred_c['oracle_tags_rewrite_seq'] = []
        for t in p['oracle_tags_rewrite_seq_prediction']:
            _pred_c['oracle_tags_rewrite_seq'].append(t)
            if t == '@end@': break
        _pred_c['oracle_tags_rewritten_question'] = p['oracle_tags_rewritten_question']
        _pred_c['oracle_tags_pred_sql'] = p['oracle_tags_pred_sql']
        _pred_c['oracle_tags_score'] = p['oracle_tags_score']
        
        _pred_c['gold_sql'] = c['query']
        
        _pred_d.append(_pred_c)

        pred_idx += 1
    
    test_pred_dataset.append(_pred_d)

len(test_pred_dataset), sum([len(d) for d in test_pred_dataset])

In [ ]:
with open('./output/test-prediction-{}.json'.format(VERSION), 'w') as f:
    json.dump(test_pred_dataset, f, indent=4)

In [ ]:
# Load dataset file with predictions 

with open('./output/pred-{}.json'.format(VERSION), 'r') as f:
    test_pred_dataset = json.load(f)
len(test_pred_dataset), sum([len(d) for d in test_pred_dataset])

In [ ]:
test_pred_dataset[0][0].keys()

In [ ]:
# Analysis 
orig_dev_preds_path = './output/dev_output.txt'

with open(orig_dev_preds_path, 'r') as f:
    orig_dev_preds = [l.strip() for l in f.readlines()]

len(orig_dev_preds)

In [ ]:
# for d in test_dataset[5::40]:
#     print('DB:', d[0]['db_id'])
#     print('ASR question:\t\t', d[0]['question'])
#     print('Rewritten question:\t', d[0]['rewritten_question'])
#     print('Gold question:\t\t', d[0]['gold_question'])
#     print('ASR-q Pred SQL:\t\t', d[0]['ratsql_pred_sql'])
#     print('Rewritten-q Pred SQL:\t', d[0]['pred_sql'])
#     print('Gold-q Pred SQL:\t', orig_dev_preds[d[0]['original_id']])
#     print('Gold SQL:\t\t', d[0]['query'])
#     print('Score:', d[0]['score'])
#     print('-'*50)

In [ ]:
test_pred_samples = [c for d in test_pred_dataset for c in d]
for i, c in list(enumerate(test_pred_samples))[8::88]:
    print('-'*30, 'ID = {}'.format(i), '-'*30)
    print('ASR question:\t\t', c['ASR_question'])
    print('Rewritten question:\t', c['rewritten_question'])
    print('Gold question:\t\t', c['gold_question'])
    print('Rewritten-q Pred SQL:\t', c['pred_sql'])
    print('Gold-q Pred SQL:\t', c['gold_question_pred_sql'])
    print('Gold SQL:\t\t', c['gold_sql'])
    print('Score:', c['score'])
    

In [ ]:
inspect_ids = [8, 96, 272, 448, 1416, 1592, 1680, 1856, 2120, 2296, 2384, 2560, 2824]

#### Seq2seq

In [34]:
## Batch evaluating 

VERSION_LIST = ['3.12.1.1', '3.12.1.2']
HUMAN_TEST = False
ASR = 'Amazon'

for VERSION in VERSION_LIST:
    print(f'Evaluating ({ASR}){" (human test)" if HUMAN_TEST else ""}: VERSION = {VERSION}')
    
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            rewriter_s2s_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/output-{VERSION}.json'
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
        else:
            rewriter_s2s_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{VERSION}.json'
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        # human test 
        rewriter_s2s_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

    
    with open(rewriter_s2s_pred_path, 'r') as f:
        rewriter_preds = [json.loads(l) for l in f.readlines()]
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
    with open(orig_dev_path, 'r') as f:
        orig_dev_dataset = json.load(f)
        
    # Quick evaluation: only using the 1st ASR candidate

    ref_list = []
    hyp_list = []
    
    pred_idx = 0

    for d in tqdm(test_dataset):
        if len(d) == 0:
            continue

        c = d[0]

        p = rewriter_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

        _db_id = o['db_id']

        # _tags = p['tags']
        # _rewrite_seq = p['rewrite_seq_prediction']
        # _question_toks = c['question_toks']

        # _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        # _rewritten_question = ' '.join(_rewritten_question_toks)

        _rewritten_question = ' '.join(p['s2s_prediction'])

        if _rewritten_question == '':
            print(f'_rewritten_question is empty')
            _pred_sql = ''
            _gold_sql = c['query']
            _exact = _score = _exec = 0
        else:
            _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
            _gold_sql = c['query']
            _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score
        c['exact'] = p['exact'] = _exact
        c['exec'] = p['exec'] = _exec
        
        # For BLEU 
        _rewritten_question_toks = [_t.lower() for _t in p['s2s_prediction']]
        _question_toks = [_t.lower() for _t in c['question_toks']]
        _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

        ref_list.append([_gold_question_toks])
        hyp_list.append(_rewritten_question_toks)

        pred_idx += len(d)

    # Only using the 1st candidate to rewrite 
    _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
    _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
    _avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

    ## Std-dev (1st cand only)
    _std_1st = np.std([d[0]['score'] for d in test_dataset])
    
    ## BLEU 
    _bleu = corpus_bleu(list_of_references=ref_list,
                        hypotheses=hyp_list)
    
    print(f'VERSION {VERSION}:')
    print(f'avg = {_avg_1st:.4f} (std = {_std_1st:.4f})')
    print(f'avg_exact = {_avg_exact_1st:.4f}')
    print(f'avg_exec = {_avg_exec_1st:.4f}')
    print(f'BLEU = {_bleu:.4f}')
    print()
    
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/ratsql-test-save/{VERSION}.json'
        else:
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
    else:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}.json'
        
    with open(test_output_path, 'w') as f:
        json.dump(test_dataset, f, indent=4)


Evaluating (Amazon): VERSION = 3.12.1.1



VERSION 3.12.1.1:
avg = 0.7410 (std = 0.3078)
avg_exact = 0.4826
avg_exec = 0.3638
BLEU = 0.8532

Evaluating (Amazon): VERSION = 3.12.1.2


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

VERSION 3.12.1.2:
avg = 0.7478 (std = 0.3037)
avg_exact = 0.4899
avg_exec = 0.3675
BLEU = 0.8566



In [ ]:
# Single version, not used 

VERSION = '3.3.0.0'

rewriter_s2s_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{VERSION}.json'
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'


In [ ]:
with open(rewriter_s2s_pred_path, 'r') as f:
    rewriter_preds = [json.loads(l) for l in f.readlines()]
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(rewriter_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

In [ ]:
rewriter_preds[0].keys()

In [ ]:
for p in rewriter_preds[3::300]:
    print(p['question'])
    print(' '.join(p['s2s_prediction']))
    print(' '.join(p['gold_rewrite_seq_s2s'][1:-1]))
    print('-'*50)

In [ ]:
# Quick evaluation: only using the 1st ASR candidate

pred_idx = 0

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
    
    p = rewriter_preds[pred_idx]
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

    _db_id = o['db_id']

    # _tags = p['tags']
    # _rewrite_seq = p['rewrite_seq_prediction']
    # _question_toks = c['question_toks']

    # _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
    # _rewritten_question = ' '.join(_rewritten_question_toks)
    
    _rewritten_question = ' '.join(p['s2s_prediction'])
    
    if _rewritten_question == '':
        print(f'_rewritten_question is empty')
        _pred_sql = ''
        _gold_sql = c['query']
        _score = 0
    else:
        _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
        _gold_sql = c['query']
        _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

    c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
    c['pred_sql'] = p['pred_sql'] = _pred_sql
    p['gold_sql'] = _gold_sql
    c['score'] = p['score'] = _score

    pred_idx += len(d)

    

In [ ]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))

In [ ]:
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

# with open(test_output_path, 'w') as f:
#     json.dump(test_dataset, f, indent=4)

In [ ]:
# Actual (full) evaluation process 

pred_idx = 0

for d in tqdm(test_dataset):
    for c in d:
        p = rewriter_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        pred_idx += 1
        if 'score' in c:
            continue  # already inferred  
        
        _db_id = o['db_id']

        _rewritten_question = ' '.join(p['s2s_prediction'])
        _pred_result = Question(_rewritten_question, _db_id)
        
        _gold_sql = c['query']
        
        if len(_pred_result) == 0:
            print(_db_id, _rewritten_question, '-- no predictiction')
            _pred_sql = ''
            _score = 0
        else:
            _pred_sql = _pred_result[0]['inferred_code']
            _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
        
        c['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score

        


In [ ]:
# Using all the candidates to rewrite 
print(sum([p['score'] for p in rewriter_preds]) / len(rewriter_preds))
print(sum([c['score'] for d in test_dataset for c in d]) / sum([len(d) for d in test_dataset]))

# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))

In [ ]:
# Merge results in a single dataset obj 

test_pred_dataset = []

pred_idx = 0

for d in tqdm(test_dataset):
    _pred_d = []
    
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _pred_c = dict()
        
        _pred_c['ASR_question'] = p['question']
        _pred_c['ASR_question_pred_sql'] = c['ratsql_pred_sql']
        
        _pred_c['gold_question'] = c['gold_question']
        # _pred_c['gold_question_pred_sql'] = orig_dev_preds[c['original_id']]
        
        _pred_c['rewritten_question'] = p['s2s_prediction']
        _pred_c['pred_sql'] = p['pred_sql']
        _pred_c['score'] = p['score']
        
        _pred_c['gold_sql'] = c['query']
        
        _pred_d.append(_pred_c)

        pred_idx += 1
    
    test_pred_dataset.append(_pred_d)

len(test_pred_dataset), sum([len(d) for d in test_pred_dataset])

In [ ]:
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

# with open(test_output_path, 'w') as f:
#     json.dump(test_dataset, f, indent=4)

In [ ]:
test_samples = [c for d in test_dataset for c in d]
for i, c in list(enumerate(test_samples))[8::88]:
    print('-'*30, 'ID = {}'.format(i), '-'*30)
    print('ASR question:\t\t', c['question'])
    print('Rewritten question:\t', c['rewritten_question'])
    print('Gold question:\t\t', c['gold_question'])
    print('Rewritten-q Pred SQL:\t', c['pred_sql'])
#     print('Gold-q Pred SQL:\t', c['gold_question_pred_sql'])
    print('Gold SQL:\t\t', c['query'])
    print('Score:', c['score'])
    

### Temp

In [ ]:
rewriter_ILM_preds[0].keys()

In [ ]:
test_dataset[0][0].keys()

In [ ]:
orig_dev_dataset[0].keys()

In [ ]:
# Rewrite_seq postprocessing, to get the rewritten question 

_idx = 154

_tags = rewriter_ILM_preds[_idx]['tags_prediction']
_rewrite_seq = rewriter_ILM_preds[_idx]['rewrite_seq_prediction']
_question_toks = rewriter_ILM_preds[_idx]['question'].split(' ')
_tags, _rewrite_seq, _question_toks

postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)

In [ ]:
db = 'concert_singer'
g_str = 'SELECT count(*) FROM singer'
p_str = "SELECT Count(DISTINCT singer.Name) FROM singer WHERE singer.Name = 'terminal'"

db, p_str, g_str, EvaluateSQL(p_str, g_str, db)

In [107]:
with open('/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/train/train_reranker.json') as f:
    _train = json.load(f)
len(_train), sum([len(d) for d in _train])

(7000, 41112)

In [108]:
del _train

In [109]:
%load_ext memory_profiler

In [110]:
%memit

peak memory: 623.44 MiB, increment: -2.01 MiB
